Objective:
My objevtive for the machine learning project is to create a Recurrent Neural Network (RNN) that can generate (somewhat) legible poetry based on a collection of poems. To do so, I will clean the data and cut up groups of words into sets of inputs for the network to take. The size of the input i.e. the number of words is yet to be determined considering we want to have a large enough input size to gather important features like rhyme schemes and context for the LSTM cell to remember, but we don't want to be too large in our input size that training is too slow. 

Instead of feeding in actual words, I will encode each word into an identifier, making the unique set of these identifiers as the Y vector for our supervised learning to take place. The vocabulary vector, our Y vector, will be one hot encoded for the word that was actually present, which the LSTM cell will attempt at predicting for each time step. 

link to data : https://www.kaggle.com/ishnoor/poetry-analysis-with-machine-learning

In [1]:
import pandas as pd
import numpy as np

poems = pd.read_csv("all.csv")

In [2]:
poems['length'] = 0
for i in range(len(poems)):
    poems['length'][i] = len(poems['content'][i])

/Users/samburtch/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
poems = poems.sort_values(by='length') #Sort by length of poem
poems = poems[14:len(poems)-5] # Delete tails on both sides
poems = poems[poems['content'].str.contains('Published')==False]# Eliminate non-poems with 'Published'
print(len(poems))
poems = poems[poems['content'].str.contains('from Selected Poems')==False]# Eliminate non-poems with 'from Selected Poems'
print(len(poems))
poems = poems[poems['content'].str.contains('Collected Poems')==False]# Eliminate non-poems with 'from Collected Poems'
print(len(poems))
for ind, row in poems.iterrows():
    if row['author'] in row['content'].upper() or str(row['poem name']) in row['content']:
        poems = poems.drop([ind])

print(len(poems))
        
poems[:10]
# Work in progress

552
536
518
455


,author,content,poem name,age,type,length
446,CARL SANDBURG,The fog comes\r\non little cat feet.\r\n\r\nIt...,Fog,Modern,Nature,115
98,EDMUND SPENSER,"New yeare forth looking out of Janus gate,\r\n...","Amoretti IV: ""New yeare forth looking out of J...",Renaissance,Love,121
45,QUEEN ELIZABETH I,"No crooked leg, no bleared eye,\r\nNo part def...",Written in her French Psalter,Renaissance,Nature,126
518,EZRA POUND,Suddenly discovering in the eyes of the very b...,"Pagani's, November 8",Modern,Love,156
510,WILLIAM BUTLER YEATS,Wine comes in at the mouth\r\nAnd love comes i...,A Drinking Song,Modern,Love,180
324,IVOR GURNEY,Only the wanderer\r\n Knows England's graces...,Song [Only the wanderer],Modern,Mythology & Folklore,190
366,IVOR GURNEY,Only the wanderer\r\n Knows England's graces...,Song [Only the wanderer],Modern,Nature,190
435,HART CRANE,Notes:\r\nThen you shall see her truly--your b...,from The Bridge: The Dance,Modern,Nature,195
383,CARL SANDBURG,The sea-wash never ends.\r\nThe sea-wash repea...,Sea-Wash,Modern,Nature,200
503,LOUISE BOGAN,Now that I know\r\nThat passion warms little\r...,Knowledge,Modern,Love,205


In [10]:
num_poems = len(poems)
poem = poems['content'][:num_poems]
poem = poem.reset_index(drop=True)
temp = ''
for i in range(num_poems):
    temp += poem[i] + '\r\n'
poem = temp
poem = poem.replace('\r\n\r\n', '\r\n')
#poem = poem.replace('\r\n', ' ')
#poem = poem.replace('\'', '')
import re
poem = re.sub(' +',' ',poem)
poem = re.findall(r"[\w']+|[.,!?;]",poem)
words = list(set(poem))
vocab_size = len(words)
print(vocab_size)


11431


RNN Model:

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

In [ ]:
poems[532]